In [ ]:
import os
import requests as rq
import pandas as pd
import numpy as np
import time
import json
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select 
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm,trange
from PIL import Image

In [9]:
with open("cinema.json") as f:    # 前面爬到的影城ID 對應表
    cinema = json.load(f)    

In [2]:
Select_Box = []

In [20]:
driver = webdriver.Chrome(ChromeDriverManager().install()) # install driver
driver.get("https://www.vscinemas.com.tw/ShowTimes")
select = Select(driver.find_element("name",'CinemaNameTWInfoF'))
option_default = True;
for op in select.options:
    if option_default:
        option_default = False
        continue
    else:
        print(re.sub(r'\(.*\)', '', op.text)

C:\Users\User\AppData\Local\Temp\ipykernel_12600\4079977351.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) # install driver


[<selenium.webdriver.remote.webelement.WebElement (session="f7b0c406a9c15813c018d8360abe9631", element="76f0c1f2-35c9-4cb8-9a2a-15bcb4002121")>, <selenium.webdriver.remote.webelement.WebElement (session="f7b0c406a9c15813c018d8360abe9631", element="c46051a9-2dc1-4f96-9ec5-035ccbf3f0ef")>, <selenium.webdriver.remote.webelement.WebElement (session="f7b0c406a9c15813c018d8360abe9631", element="b7de3d36-ffcb-4c7f-a1b6-0d19b1c2f883")>, <selenium.webdriver.remote.webelement.WebElement (session="f7b0c406a9c15813c018d8360abe9631", element="5432c2ed-8e42-405b-9de3-ae841b9c0fa8")>, <selenium.webdriver.remote.webelement.WebElement (session="f7b0c406a9c15813c018d8360abe9631", element="e9240779-9df6-48ee-9217-1574d5681c01")>, <selenium.webdriver.remote.webelement.WebElement (session="f7b0c406a9c15813c018d8360abe9631", element="65e3ff92-5598-463a-a3b2-82cab65ec0da")>, <selenium.webdriver.remote.webelement.WebElement (session="f7b0c406a9c15813c018d8360abe9631", element="9c0ae62f-7894-4afc-af47-02d96a8d

NameError: name 'option_defaulttoi' is not defined

In [ ]:
for i in range(1,2): # 要改成range(1,25)
    if i == 1:
        select.select_by_index(i) #   首頁跟之後select放的地方不同 
    else:
        select = Select(driver.find_element("name",'CinemaNameTWInfoS')) 
        select.select_by_index(i)
    time.sleep(2) # for delay
    content  = BeautifulSoup(driver.page_source,"html.parser")
    Title = content.select('div #ShowTimeSecondInfo div.col-xs-12 strotng.col-xs-12.LangTW.MovieName')
    Time = content.select_one('div #ShowTimeSecondInfo').text.replace(" ","").replace("\n\n","\n").replace('\n'," ").replace("  "," ").replace(" ","\n")
    print(Time) 

In [16]:
print(ShowTime)

{'台北信義威秀影城': [], 'MUVIE CINEMAS台北松仁威秀': [], 'MUVIE CINEMAS台北松仁威秀 (MUCROWN)': [], '台北京站威秀影城': [], '板橋大遠百威秀影城': [], '中和環球威秀影城': [], '林口MITSUI OUTLET PARK威秀影城': [], '林口MITSUI OUTLET PARK威秀影城 (Mappa)': [], '桃園統領威秀影城': [], '新竹大遠百威秀影城': [], '新竹大遠百威秀影城 (GC)': [], '新竹巨城威秀影城': [], '頭份尚順威秀影城': [], '台中大遠百威秀影城': [], '台中Tiger City威秀影城': [], '台中Tiger City威秀影城 (GC)': [], '台中大魯閣新時代威秀影城': [], '台南大遠百威秀影城': [], '台南FOCUS威秀影城': [], '台南南紡威秀影城': [], '台南南紡威秀影城 (GC)': [], '高雄大遠百威秀影城': [], '高雄大遠百威秀影城 (GC)': [], '花蓮新天堂樂園威秀影城': []}


In [21]:
print(Title)

[<strong class="col-xs-12 LangTW MovieName">
            (MUCROWN)黑亞當 (保護級)
        </strong>, <strong class="col-xs-12 LangTW MovieName">
            (MUCROWN)黑豹2：瓦干達萬歲 (保護級)
        </strong>]


In [75]:
reg = 0
count = 0
Name = []
Date = []
Show = []
Info = []
s = ""
name = ""
date = ""
show = ""
for time in new:
    if count == 2:
        reg = 1
        count = 0
        name = ""
    elif reg == 1 and time != '\n':
        name += time
        count = 0
    elif time == '\n':
        reg += 1
        count += 1
        if len(s) == 9:
            date = s
        elif len(s) == 5:
            show = s
            info = name[name.find(')')+1:name.rfind('(')] + " "+date+" "+show
            Info.append(info)
        s = ""
    else:
        s += time
        count = 0

In [22]:
print(new)


(MUCROWN)黑亞當(保護級)
(MUCROWN)BLACKADAM(P)
11月07日星期一
11/07Mon
13:00
16:00
19:00
22:00
11月08日星期二
11/08Tue
13:00
16:00
19:00
22:00

(MUCROWN)黑豹2：瓦干達萬歲(保護級)
(MUCROWN)BLACKPANTHERWAKANDAFOREVER(P)
11月09日星期三
11/09Wed
11:20
15:10
19:00
22:50
11月10日星期四
11/10Thu
11:20
15:10
19:00
22:50
11月11日星期五
11/11Fri
11:20
15:10
22:50
11月12日星期六
11/12Sat
16:30
23:30
11月13日星期日
11/13Sun
16:30
20:00
23:30
11月14日星期一
11/14Mon
11:20
15:10
19:00
22:50



In [77]:
for i in Info:
    print (i)

黑亞當 11月07日星期一 13:00
黑亞當 11月07日星期一 16:00
黑亞當 11月07日星期一 19:00
黑亞當 11月07日星期一 22:00
黑亞當 11月08日星期二 13:00
黑亞當 11月08日星期二 16:00
黑亞當 11月08日星期二 19:00
黑亞當 11月08日星期二 22:00
黑豹2：瓦干達萬歲 11月09日星期三 11:20
黑豹2：瓦干達萬歲 11月09日星期三 15:10
黑豹2：瓦干達萬歲 11月09日星期三 19:00
黑豹2：瓦干達萬歲 11月09日星期三 22:50
黑豹2：瓦干達萬歲 11月10日星期四 11:20
黑豹2：瓦干達萬歲 11月10日星期四 15:10
黑豹2：瓦干達萬歲 11月10日星期四 19:00
黑豹2：瓦干達萬歲 11月10日星期四 22:50
黑豹2：瓦干達萬歲 11月11日星期五 11:20
黑豹2：瓦干達萬歲 11月11日星期五 15:10
黑豹2：瓦干達萬歲 11月11日星期五 22:50
黑豹2：瓦干達萬歲 11月12日星期六 16:30
黑豹2：瓦干達萬歲 11月12日星期六 23:30
黑豹2：瓦干達萬歲 11月13日星期日 16:30
黑豹2：瓦干達萬歲 11月13日星期日 20:00
黑豹2：瓦干達萬歲 11月13日星期日 23:30
黑豹2：瓦干達萬歲 11月14日星期一 11:20
黑豹2：瓦干達萬歲 11月14日星期一 15:10
黑豹2：瓦干達萬歲 11月14日星期一 19:00
黑豹2：瓦干達萬歲 11月14日星期一 22:50
